In [1]:
import functools

import numpy as np
import tensorflow as tf

In [2]:
# Download Data using tf.keras.utils.get_file

TRAIN_DATA_URL = "https://storage.googleapis.com/tf-datasets/titanic/train.csv"
TEST_DATA_URL = "https://storage.googleapis.com/tf-datasets/titanic/eval.csv"

train_file_path = tf.keras.utils.get_file("train.csv", TRAIN_DATA_URL)
test_file_path = tf.keras.utils.get_file("eval.csv", TEST_DATA_URL)

16384/13049 [=====================================] - 0s 0us/step


In [3]:
# Make numpy values easier to read.

np.set_printoptions(precision = 3, suppress = True)

In [4]:
LABEL_COLUMN = 'survived'
LABELS = [0, 1]

In [5]:
# Read CSV Files onto a dataset
# tf.data.experimental.make_csv_dataset returns tensors as a tuple (features, labels)

def get_dataset(file_path, **kwargs):
  dataset = tf.data.experimental.make_csv_dataset(
      file_path,
      batch_size = 5, 
      label_name=LABEL_COLUMN,
      na_value = "?",
      num_epochs = 1,
      ignore_errors = True, 
      **kwargs)
  return dataset

raw_train_data = get_dataset(train_file_path)
raw_test_data = get_dataset(test_file_path)

In [6]:
# tf.data.dataset class, take(count) function returns a dataset with count number of elements

def show_batch(dataset):
  for batch, label in dataset.take(1):
    for key, value in batch.items():
      print("{:20s}: {}".format(key,value.numpy()))

In [7]:
# Mixed Dataset (dtypes) 

show_batch(raw_train_data)

sex                 : [b'male' b'male' b'female' b'male' b'male']
age                 : [34. 28. 28. 43. 19.]
n_siblings_spouses  : [0 0 0 0 0]
parch               : [0 0 0 0 0]
fare                : [6.496 8.05  8.05  8.05  6.75 ]
class               : [b'Third' b'Third' b'Third' b'Third' b'Third']
deck                : [b'unknown' b'unknown' b'unknown' b'unknown' b'unknown']
embark_town         : [b'Southampton' b'Southampton' b'Southampton' b'Southampton' b'Queenstown']
alone               : [b'y' b'y' b'y' b'y' b'y']


In [8]:
# Working with continous data

# Pack into a vector before passing into the model

In [9]:
SELECT_COLUMNS = ['survived', 'age', 'n_siblings_spouses', 'parch', 'fare']
DEFAULTS = [0, 0.0, 0.0, 0.0, 0.0]
temp_dataset = get_dataset(train_file_path, 
                           select_columns=SELECT_COLUMNS,
                           column_defaults = DEFAULTS)

show_batch(temp_dataset)

age                 : [ 2. 28. 39. 28. 21.]
n_siblings_spouses  : [1. 8. 0. 0. 2.]
parch               : [1. 2. 0. 0. 2.]
fare                : [ 26.     69.55   13.      7.75  262.375]


In [10]:
# Use iter to make it into a generator

example_batch, labels_batch = next(iter(temp_dataset)) 

In [11]:
# Pack into a vector

def pack(features, label):
  return tf.stack(list(features.values()), axis = -1), label

In [12]:
packed_dataset = temp_dataset.map(pack)

for features, labels in packed_dataset.take(1):
  print(features.numpy())
  print()
  print(labels.numpy())

[[35.     0.     0.    26.   ]
 [31.     0.     0.    10.5  ]
 [28.     0.     0.     7.775]
 [28.     0.     0.    47.1  ]
 [45.     0.     0.     7.75 ]]

[0 0 0 0 0]
